In [1]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mask = np.load(r"C:\Users\DU\aman_fastmri\Data\mask_4x_320_random.npy")  # Shape: (1, 320, 320)
print("og shape:", mask.shape)

# # Use np.tile to reshape it to (1, 320, 320, 1)
# # var_sampling_mask = np.tile(var_sampling_mask[..., np.newaxis], (1, 1, 1, 1))  # Final shape: (1, 320, 320, 1)
# mask = np.tile(mask, (1, 320, 1, 2))  # tile height=320 times

# # Confirm final shape
# print("New shape:", mask.shape) 
# mask_for_plot = np.squeeze(mask[...,0])  # Shape: (320, 320)

# # Plot
# plt.figure(figsize=(5, 5))
# plt.imshow(mask_for_plot, cmap='gray')
# plt.title("Tiled Sampling Mask (320x320)")
# plt.axis('off')
# plt.show()

og shape: (1, 1, 320, 1)


In [6]:
import h5py
import numpy as np
import tensorflow as tf

class MRISliceGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_list, batch_size=4, shuffle=True, mask=None):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.mask = mask  # Shape: (1, 320, 320, 2)
        self.slice_index_map = []
        self._build_index()

    def _build_index(self):
        for file_idx, file_path in enumerate(self.file_list):
            with h5py.File(file_path, 'r') as f:
                num_slices = f['image_under'].shape[0]
                for slice_idx in range(num_slices):
                    self.slice_index_map.append((file_idx, slice_idx))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.slice_index_map) / self.batch_size))

    def __getitem__(self, index):
        batch_map = self.slice_index_map[index * self.batch_size:(index + 1) * self.batch_size]

        input_img_batch = []
        target_img_batch = []
        input_kspace_batch = []

        for file_idx, slice_idx in batch_map:
            with h5py.File(self.file_list[file_idx], 'r') as f:
                #input_img = f['image_under'][slice_idx]       # shape: [H, W, 2]
                target_img = f['image_full'][slice_idx]       # shape: [H, W, 2]
                input_kspace = f['kspace_under'][slice_idx]   # shape: [H, W, 2]

                #input_img_batch.append(input_img)
                target_img_batch.append(target_img)
                input_kspace_batch.append(input_kspace)

        #x_img = np.stack(input_img_batch, axis=0)
        x_kspace = np.stack(input_kspace_batch, axis=0)
        y_batch = np.stack(target_img_batch, axis=0)

        if self.mask is not None:
            actual_batch_size = len(x_kspace)

            # mask must be (1, 1, W, 1)
            if self.mask.shape != (1, 1, 320, 1):
                raise ValueError("Mask must have shape (1, 1, W, 1)")
        
            # Tile ONLY along batch dimension
            mask_batch = np.tile(self.mask, (actual_batch_size, 1, 1, 1))
        
        return [x_kspace, mask_batch], y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.slice_index_map)


In [7]:
val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"

In [8]:
import h5py
import numpy as np
import glob
import os

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))


half_val = len(kspace_files_list_val) 

kspace_files_list_val = kspace_files_list_val[:half_val]


val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False,mask=mask)
# train_gen = MRISliceGenerator(kspace_files_list_train,batch_size=16, shuffle=True)
# val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False)


print(len(val_gen))  


1784


In [9]:
%run ./TEID_Net.ipynb


Model: "CascadedTEIDNet_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 kspace_undersampled (InputLaye  [(None, 320, 320, 2  0          []                               
 r)                             )]                                                                
                                                                                                  
 sampling_mask (InputLayer)     [(None, 1, 320, 1)]  0           []                               
                                                                                                  
 cascaded_teid_net (CascadedTEI  (None, 320, 320, 2)  2521475    ['kspace_undersampled[0][0]',    
 DNet)                                                            'sampling_mask[0][0]']          
                                                                              

In [4]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

import numpy as np

def count_parameters_millions(model):
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_variables])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_variables])
    total = trainable + non_trainable
    return (
        total / 1e6,
        trainable / 1e6,
        non_trainable / 1e6
    )

total_M, trainable_M, non_trainable_M = count_parameters_millions(model)

print("\n" + "=" * 40)
print(f"Total parameters:       {total_M:.3f} M")
print(f"Trainable parameters:   {trainable_M:.3f} M")
print(f"Non-trainable params:   {non_trainable_M:.3f} M")
print("=" * 40)
import tensorflow as tf
def compute_flops_multi_input(model, input_shapes):
    """
    input_shapes: list of tuples
      e.g. [(1, H, W, 2), (1, 1, W, 1)]
    """

    @tf.function
    def forward(*inputs):
        return model(inputs)

    concrete_func = forward.get_concrete_function(
        *[tf.TensorSpec(shape, tf.float32) for shape in input_shapes]
    )

    frozen_func = convert_variables_to_constants_v2(concrete_func)
    graph_def = frozen_func.graph.as_graph_def()

    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name="")

        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

        flops = tf.compat.v1.profiler.profile(
            graph=graph,
            run_meta=run_meta,
            cmd="op",
            options=opts
        )

    return flops.total_float_ops
H, W = 320, 320

input_shapes = [
    (1, H, W, 2),   # kspace_undersampled
    (1, 1, W, 1)    # sampling_mask
]

flops = compute_flops_multi_input(model, input_shapes)

print(f"FLOPs (single forward pass): {flops / 1e9:.2f} GFLOPs")



Total parameters:       2.521 M
Trainable parameters:   2.521 M
Non-trainable params:   0.000 M
FLOPs (single forward pass): 274.85 GFLOPs


In [10]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ============================================================
# Directory Setup
# ============================================================
save_dir = "./SavedModels_TEID_Net_lr_full"
H, W = 320, 320
EPOCHS = 20
LEARNING_RATE = 1e-4
model = build_cascaded_teid_model(
    H=H,
    W=W,
    num_cascades=5,
    channels=64
)

# ============================================================
# Optimizer & Compile
# ============================================================
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="mae")

# ============================================================
# Load Initial Weights (Optional Resume)
# ============================================================
if tf.train.latest_checkpoint(save_dir):
    model.load_weights(tf.train.latest_checkpoint(save_dir))
    print("✅ Loaded latest checkpoint")
else:
    print("ℹ️ No checkpoint found. Training from scratch.")

✅ Loaded latest checkpoint


In [6]:
import os
import time
import glob
import psutil
import numpy as np
import h5py
from tqdm import tqdm
import tensorflow as tf
from keras_flops import get_flops

# ============================================================
# CONFIGURATION
# ============================================================
VAL_FOLDER = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
WARMUP_SLICES = 10
NUM_TIMING_SLICES = 100   # increase if needed

# ============================================================
# FILE LIST
# ============================================================
file_paths = sorted(glob.glob(os.path.join(VAL_FOLDER, "*.h5")))

# ============================================================
# MODEL & MASK MUST BE LOADED
# ============================================================
# model = ...
# mask  = ...   # shape (1, H, W, 1) or (1, H, W, 2)

assert model is not None, "Model is not loaded."
assert mask is not None, "Mask is not loaded."

# ============================================================
# PARAMETER COUNT
# ============================================================
num_params = model.count_params()

# ============================================================
# FLOPs (PER SLICE, BATCH SIZE = 1)
# ============================================================
flops = get_flops(model, batch_size=1)

# ============================================================
# MEMORY HELPERS
# ============================================================
process = psutil.Process(os.getpid())

def cpu_memory_mb():
    return process.memory_info().rss / (1024 ** 2)

def gpu_memory_mb():
    info = tf.config.experimental.get_memory_info("GPU:0")
    return info["peak"] / (1024 ** 2)

# ============================================================
# LATENCY / THROUGHPUT MEASUREMENT
# ============================================================
def measure_latency(device):

    latencies = []

    with tf.device(device):

        # -----------------------------
        # WARM-UP
        # -----------------------------
        for file in file_paths[:1]:
            with h5py.File(file, "r") as f:
                kspace_under = f["kspace_under"][:]

            for s in range(min(WARMUP_SLICES, kspace_under.shape[0])):
                slice_k = kspace_under[s:s+1]
                slice_m = mask
                _ = model([slice_k, slice_m], training=False)

        # -----------------------------
        # TIMED INFERENCE
        # -----------------------------
        count = 0
        for file in tqdm(file_paths, desc=f"Timing on {device}"):

            with h5py.File(file, "r") as f:
                kspace_under = f["kspace_under"][:]

            for s in range(kspace_under.shape[0]):

                if count >= NUM_TIMING_SLICES:
                    break

                slice_k = kspace_under[s:s+1]   # (1, H, W, 2)
                slice_m = mask                  # (1, H, W, 1 or 2)

                assert slice_k.shape[0] == 1

                start = time.perf_counter()
                _ = model([slice_k, slice_m], training=False)

                # 🔑 GPU synchronization
                if "GPU" in device:
                    tf.config.experimental.get_memory_info("GPU:0")

                end = time.perf_counter()

                latencies.append(end - start)
                count += 1

            if count >= NUM_TIMING_SLICES:
                break

    latencies = np.array(latencies)

    mean_s = latencies.mean()
    median_s = np.median(latencies)

    return {
        "mean_s": mean_s,
        "median_s": median_s,
        "std_s": latencies.std(),
        "slices_per_sec": 1.0 / mean_s
    }

# ============================================================
# CPU BENCHMARK
# ============================================================
cpu_mem_before = cpu_memory_mb()
cpu_latency = measure_latency("/CPU:0")
cpu_mem_after = cpu_memory_mb()
cpu_mem_peak = cpu_mem_after - cpu_mem_before

# ============================================================
# GPU BENCHMARK (IF AVAILABLE)
# ============================================================
gpu_latency = None
gpu_mem_peak = None
gpu_name = None

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.reset_memory_stats("GPU:0")
    gpu_name = tf.config.experimental.get_device_details(gpus[0])["device_name"]
    gpu_latency = measure_latency("/GPU:0")
    gpu_mem_peak = gpu_memory_mb()

# ============================================================
# FINAL REPORT
# ============================================================
print("\n" + "=" * 70)
print("MODEL EFFICIENCY REPORT (BATCH SIZE = 1)")
print("=" * 70)

print(f"Parameters: {num_params / 1e6:.2f} M")
print(f"FLOPs:      {flops / 1e9:.2f} GFLOPs (per slice)")

print("\n--- CPU Inference ---")
print(f"Latency:    {cpu_latency['mean_s']:.2f} s / slice")
print(f"Throughput:{cpu_latency['slices_per_sec']:.3f} slices/sec")
print(f"Memory:     {cpu_mem_peak:.2f} MB")

if gpu_latency:
    print("\n--- GPU Inference ---")
    print(f"GPU:        {gpu_name}")
    print(f"Latency:    {gpu_latency['mean_s']:.2f} s / slice")
    print(f"Throughput:{gpu_latency['slices_per_sec']:.3f} slices/sec")
    print(f"Peak VRAM:  {gpu_mem_peak:.2f} MB")
else:
    print("\nGPU not available.")

print("=" * 70)


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Timing on /GPU:0:   1%|▌                                                             | 2/199 [02:20<3:50:52, 70.32s/it]


MODEL EFFICIENCY REPORT (BATCH SIZE = 1)
Parameters: 2.52 M
FLOPs:      275.22 GFLOPs (per slice)

--- CPU Inference ---
Latency:    8.09 s / slice
Throughput:0.124 slices/sec
Memory:     12.40 MB

--- GPU Inference ---
GPU:        NVIDIA RTX A5000
Latency:    1.41 s / slice
Throughput:0.711 slices/sec
Peak VRAM:  226.76 MB


In [14]:
import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# # Path to validation folder
# # val_folder = "F:/denoised_preprocessed_h5_val"

# val_folder = r"E:\fastmri\val_norm"
# # val_folder = r"D:\val_norm"

# # val_folder = r"G:\val_norm\val_norm"
# # files = sorted([os.path.join(val_folder, f) for f in os.listdir(val_folder) if f.endswith(".h5")])
# kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
file_paths = kspace_files_list_val


# ----------------------
# HELPERS
# ----------------------
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )

# ----------------------
# STORAGE
# ----------------------
ssim_list = []
psnr_list = []
nmse_list = []

# ----------------------
# PROCESSING
# ----------------------
for file in tqdm(pd_files, desc="Processing volumes"):
    with h5py.File(file, 'r') as f:
        kspace_under = f["kspace_under"][:]   # (S, H, W, 2)
        image_full   = f["image_full"][:]     # (S, H, W, 2)
        max_val = float(f["max_val_full_image"][0])

    num_slices = kspace_under.shape[0]

    # --------------------------------------------------
    # TILE MASK FOR THIS VOLUME
    # --------------------------------------------------
    mask_batch = np.tile(mask, (num_slices, 1, 1, 1))

    # --------------------------------------------------
    # MODEL INFERENCE
    # --------------------------------------------------
    pred = model.predict(
        [kspace_under, mask_batch],
        batch_size=1,
        verbose=0
    )

    
    image_full *= max_val
    
    pred *= max_val  # Scale predicted output to original intensity range
    #psnr_val = peak_signal_noise_ratio(image_full, pred, data_range=max_val)

    # Convert to complex and get magnitude
    gt_mag = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # Volume-wise PSNR and NMSE
    psnr_val = peak_signal_noise_ratio(gt_mag, pred_mag, data_range=max_val)
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())

    psnr_list.append(psnr_val)
    nmse_list.append(nmse_val)

    # Slice-wise SSIM
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)
        ssim_list.append(ssim_val)

# ----------------------
# REPORT
# ----------------------
print("\n" + "=" * 40)
print(f"PSNR : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")

print("=" * 40)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 100/100 [11:18<00:00,  6.79s/it]


PSNR : 33.9939 ± 2.5453 dB
NMSE (Mag, volume): 0.015349 ± 0.007232
SSIM (Mag, slice):  0.8132 ± 0.0676


In [13]:
print("\n" + "=" * 40)
print(f"PSNR : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.4f} ± {np.std(nmse_list):.4f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")

print("=" * 40)


PSNR : 32.6425 ± 2.5847 dB
NMSE (Mag, volume): 0.0293 ± 0.0128
SSIM (Mag, slice):  0.7691 ± 0.0774


In [11]:
pd_files = []
pdfs_files = []

for f in kspace_files_list_val:
    if "PDFS" in f:
        pdfs_files.append(f)
    else:
        pd_files.append(f)

print(f"PD volumes: {len(pd_files)}")
print(f"PDFS volumes: {len(pdfs_files)}")


PD volumes: 100
PDFS volumes: 99


In [14]:
import os
import glob
import numpy as np
import h5py
import pandas as pd
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# ======================================================
# USER INPUTS (adjust if needed)
# ======================================================

val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"   # validation H5 folder
output_excel = "validation_metrics_TEID_FI.xlsx"

# model : your trained model (already loaded)
# mask  : undersampling mask of shape (1, H, W, 1) or (1, H, W, 2)

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))

# ======================================================
# HELPER FUNCTIONS
# ======================================================

def to_complex(x):
    """Convert (..., 2) real/imag to complex"""
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt,
        pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )

# ======================================================
# METRIC STORAGE
# ======================================================

volume_metrics = []   # PSNR + NMSE (per volume)
slice_metrics  = []   # SSIM (per slice)

# ======================================================
# PROCESSING LOOP
# ======================================================

for file in tqdm(kspace_files_list_val, desc="Processing volumes"):
    volume_name = os.path.basename(file)

    with h5py.File(file, "r") as f:
        kspace_under = f["kspace_under"][:]      # (S, H, W, 2)
        image_full   = f["image_full"][:]        # (S, H, W, 2)
        max_val      = float(f["max_val_full_image"][0])

    num_slices = kspace_under.shape[0]

    # --------------------------------------------------
    # Mask tiling
    # --------------------------------------------------
    mask_batch = np.tile(mask, (num_slices, 1, 1, 1))

    # --------------------------------------------------
    # Model inference
    # --------------------------------------------------
    pred = model.predict(
        [kspace_under, mask_batch],
        batch_size=1,
        verbose=0
    )

    # --------------------------------------------------
    # Rescale to original intensity
    # --------------------------------------------------
    image_full = image_full * max_val
    pred       = pred * max_val

    # --------------------------------------------------
    # Magnitude images
    # --------------------------------------------------
    gt_mag   = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # --------------------------------------------------
    # Volume-wise metrics
    # --------------------------------------------------
    psnr_val = peak_signal_noise_ratio(
        gt_mag,
        pred_mag,
        data_range=max_val
    )

    nmse_val = nmse(
        gt_mag.flatten(),
        pred_mag.flatten()
    )

    volume_metrics.append({
        "volume_name": volume_name,
        "num_slices": num_slices,
        "PSNR_dB": psnr_val,
        "NMSE": nmse_val
    })

    # --------------------------------------------------
    # Slice-wise SSIM
    # --------------------------------------------------
    for i in range(num_slices):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)

        slice_metrics.append({
            "volume_name": volume_name,
            "slice_index": i,
            "SSIM": ssim_val
        })

# ======================================================
# SAVE TO EXCEL
# ======================================================

df_volume = pd.DataFrame(volume_metrics)
df_slice  = pd.DataFrame(slice_metrics)

with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    df_volume.to_excel(writer, sheet_name="Volume_Metrics_TEID_Final", index=False)
    df_slice.to_excel(writer, sheet_name="Slice_SSIM_Final", index=False)

# ======================================================
# REPORT
# ======================================================

print("\n" + "=" * 50)
print("Evaluation complete")
print(f"Excel file saved to: {output_excel}")
print("=" * 50)

print(f"PSNR : {df_volume['PSNR_dB'].mean():.4f} ± {df_volume['PSNR_dB'].std():.4f} dB")
print(f"NMSE : {df_volume['NMSE'].mean():.6f} ± {df_volume['NMSE'].std():.6f}")
print(f"SSIM : {df_slice['SSIM'].mean():.4f} ± {df_slice['SSIM'].std():.4f}")
print("=" * 50)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 199/199 [26:24<00:00,  7.96s/it]



Evaluation complete
Excel file saved to: validation_metrics_TEID_FI.xlsx
PSNR : 33.3216 ± 2.6591 dB
NMSE : 0.022312 ± 0.012549
SSIM : 0.7913 ± 0.0759
